In [9]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import unidecode

def makeSoup(url):
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    return soup

#Prompt the user to enter the years for the scrape
start_year = input('----Input the Start Year:  ')
stop_year = input('----Input the Final Year:  ')
years = range(int(start_year),int(stop_year)+1)

output = []
for year in years:
    soup = makeSoup('https://www.basketball-reference.com/leagues/NBA_'+str(year)+'_play-by-play.html')
    posTable = soup.find('table',{'id': 'pbp_stats'})

    #Pull all Stats from table
    posData = [[td.getText() for td in posTable.findAll('tr')[1:][i].findAll('td')]
               for i in range(len(posTable.findAll('tr')[1:]))]

    #Cleaning Operations
    posData = [item for item in posData if len(item)>0] #Remove Blank Rows
    posData = [item for item in posData if item[3]!='TOT'] #Remove Cumulative Rows
    posData = [[year] + item[:11] for item in posData] #Remove Unused Columns
    for item in posData:
        item[1] = unidecode.unidecode(item[1]).replace('*','').replace('.','')#Remove Special Characters
        item[7] = item[7].replace('%','')
        item[8] = item[8].replace('%','')
        item[9] = item[9].replace('%','')
        item[10] = item[10].replace('%','')
        item[11] = item[11].replace('%','')

    #Convert list to Pandas Dataframe
    posData_df = pd.DataFrame(posData)
    output.append(posData_df)

output = pd.concat(output)
header = ['Year','Player','Pos','Age','Team','Games','MP','PG%','SG%','SF%','PF%','C%']
output.columns = header

output.to_csv('posStats.csv',index=False)

----Input the Start Year:   2021
----Input the Final Year:   2021
